In [29]:
import pandas as pd
import sqlite3 

In [30]:
connection = sqlite3.connect('../data/checking-logs.sqlite')

In [31]:
query = '''
WITH users_with_both_periods AS (
    SELECT uid
    FROM test
    WHERE labname <> 'project1'
    GROUP BY uid
    HAVING 
        SUM(CASE WHEN first_commit_ts < first_view_ts THEN 1 ELSE 0 END) > 0 AND
        SUM(CASE WHEN first_commit_ts >= first_view_ts THEN 1 ELSE 0 END) > 0
),

commit_differences AS (
    SELECT 
        t.uid,
        CASE 
            WHEN t.first_commit_ts < t.first_view_ts THEN 'before' 
            ELSE 'after' 
        END AS time_period,
        (JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24 AS hours_diff
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE 
        t.labname <> 'project1' AND
        t.uid IN (SELECT uid FROM users_with_both_periods)
)

SELECT 
    time_period AS time,
    AVG(hours_diff) AS avg_diff
FROM commit_differences
GROUP BY time_period
'''
test_results = pd.io.sql.read_sql(query, connection)
test_results

,time,avg_diff
0,after,-105.229101
1,before,-61.156438


In [32]:
query = '''
WITH users_with_both_periods AS (
    SELECT uid
    FROM control
    WHERE labname <> 'project1'
    GROUP BY uid
    HAVING 
        SUM(CASE WHEN first_commit_ts < first_view_ts THEN 1 ELSE 0 END) > 0 AND
        SUM(CASE WHEN first_commit_ts >= first_view_ts THEN 1 ELSE 0 END) > 0
),

commit_differences AS (
    SELECT 
        c.uid,
        CASE 
            WHEN c.first_commit_ts < c.first_view_ts THEN 'before' 
            ELSE 'after' 
        END AS time_period,
        (JULIANDAY(c.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24 AS hours_diff
    FROM control c
    JOIN deadlines d ON c.labname = d.labs
    WHERE 
        c.labname <> 'project1' AND
        c.uid IN (SELECT uid FROM users_with_both_periods)
)

SELECT 
    time_period AS time,
    AVG(hours_diff) AS avg_diff
FROM commit_differences
GROUP BY time_period
'''
control_results = pd.io.sql.read_sql(query, connection)
control_results

,time,avg_diff
0,after,-118.144425
1,before,-99.901295


## Ответ на вопрос : гипотеза оказалась верной и страница действительно влияет на поведение студентов?

### Да, гипотеза подтвердилась, т.к. после просмотра ленты тестовая группа стала сдавать лабы на 44 часа раньше, чем в контрольной

In [33]:
data = {
    'Group': ['Test', 'Control'],
    'Before (hours before the deadline)': [-61.16, -99.90],
    'After (hours before the deadline)': [-105.23, -118.14],
    'Difference (After - Before)': [-44.07, -18.24]
}

df = pd.DataFrame(data)
df

,Group,Before (hours before the deadline),After (hours before the deadline),Difference (After - Before)
0,Test,-61.16,-105.23,-44.07
1,Control,-99.90,-118.14,-18.24


#### отрицательные значения означают, что студенты в среднем сдавали работы раньше дедлайна

In [34]:
connection.close()